In [1]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import sklearn.decomposition
import pickle


with open('../data/schaefer.pickle', 'rb') as f:
    all_parc = pickle.load(f)

In [2]:
def utri2mat(utri):
    n = int(-1 + np.sqrt(1 + 8 * len(utri))) // 2
    iu1 = np.triu_indices(n)
    ret = np.empty((n, n))
    ret[iu1] = utri
    ret.T[iu1] = utri
    return ret

In [3]:
# nSubj = int(all_parc[500]['FC_all_vec'].shape[0] / 16)
# nFCs = int(all_parc[500]['FC_all_vec'].shape[0])
# testFCs = all_parc[500]['FC_all_vec'][::2]
# retest_FCs = all_parc[500]['FC_all_vec'][1::2]
# reordered_FCs = np.float32(np.concatenate([testFCs, retest_FCs]))
# del testFCs, retest_FCs
# all_FC = np.zeros((nFCs, 500 + 13, 500 + 13),dtype="float32")
# for i in np.arange(0, nFCs):
#     all_FC[i] = utri2mat(reordered_FCs[i])
    
# with open('../data/schaefer500.pickle', 'wb') as f:
#     pickle.dump(all_FC, f, protocol=4)

(6768, 513, 513)


In [58]:
parc = 400
with open(f'../data/schaefer{parc}.pickle', 'rb') as f:
    all_FC = pickle.load(f)
nSubj = int(all_FC.shape[0]/16)
nFCs = int(all_FC.shape[0])

## Deep Learning

In [59]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils import data
from sklearn.preprocessing import StandardScaler

# GPU is available? If so, we use it.
use_cuda = torch.cuda.is_available() 
device = torch.device("cuda:0" if use_cuda else "cpu")
if use_cuda:
    print("GPU detected. Will use GPU for training!")
    torch.backends.cudnn.benchmark = True
else:
    print("No GPU detected. Will use CPU for training.")

No GPU detected. Will use CPU for training.


In [60]:
# Create labels corresponding to all_FC 
labels = torch.tensor(np.tile(np.repeat(np.arange(0,8),nSubj),2), dtype=torch.long)
# Randomly shuffled indices for test FCs
indices = np.random.permutation(nFCs)
# Take subsets of data for training, validation, test
train_val_idx = indices[:int(0.8*nFCs)]

val_idx = train_val_idx[int(0.8*train_val_idx.shape[0]):]
train_idx = train_val_idx[:int(0.8*train_val_idx.shape[0])]
test_idx = indices[int(0.8*nFCs):]

In [61]:
train_mean = np.mean(all_FC[train_idx])
train_std = np.std(all_FC[train_idx])
train_data = torch.FloatTensor((all_FC[train_idx] - train_mean) / train_std)
val_data = torch.FloatTensor((all_FC[val_idx] - train_mean) / train_std)
test_data = torch.FloatTensor((all_FC[test_idx] - train_mean) / train_std)

In [62]:
train_data = train_data.view(train_data.shape[0], -1, train_data.shape[1], train_data.shape[2])
val_data = val_data.view(val_data.shape[0], -1, val_data.shape[1], val_data.shape[2])
test_data = test_data.view(test_data.shape[0], -1, test_data.shape[1], test_data.shape[2])

In [63]:
train_dataset = data.TensorDataset(train_data,labels[train_idx]) # create your datset
val_dataset = data.TensorDataset(val_data,labels[val_idx]) # create your datset
test_dataset = data.TensorDataset(test_data,labels[test_idx]) # create your datset

train_loader = data.DataLoader(train_dataset, batch_size=80) # create your dataloader
val_loader = data.DataLoader(val_dataset, batch_size=80) # create your dataloader
test_loader = data.DataLoader(test_dataset, batch_size=80) # create your dataloader


## NN Architecture

In [64]:
del all_FC
output_size = 8
max_epochs = 50
n_epochs_stop = 5

In [69]:
class Net(nn.Module):
    def __init__(self, nHidden):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 12, kernel_size=5)
        self.conv2_bn = nn.BatchNorm2d(12)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(nHidden, 128)
        self.fc2 = nn.Linear(128, 8)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), (3,3)))
        x = F.relu(F.max_pool2d(self.conv2_bn(self.conv2(x)), (3,3)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

hidden_dict = {100: 1200, 200: 5292, 300: 13068, 400: 23232, 500: 36300}
model = Net(hidden_dict[parc])


In [66]:
loss_fn = nn.CrossEntropyLoss()
opt = optim.SGD(model.parameters(), lr=0.001)
if use_cuda:
    model = model.cuda()
print(model)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (conv2_bn): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=1200, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=8, bias=True)
)


In [67]:
load_model = False
if load_model:
    the_model = Net()
    the_model.load_state_dict(torch.load("../trained_models/ENTER NAME HERE"))

In [68]:
history = {}
history['loss'] = []
history['val_loss'] = []
history['acc'] = []
history['val_acc'] = []
min_val_loss = np.Inf
early_stop = False
# Loop over epochs


for epoch in range(max_epochs):
    # Training
    model.train()
    train_loss         = 0.0
    num_train_correct  = 0
    num_train_examples = 0
    for local_batch, local_labels in train_loader:
        # Transfer to GPU
        if use_cuda:
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)



        opt.zero_grad()
        output = model(local_batch)
        loss = loss_fn(output, local_labels)
        loss.backward()
        opt.step()

        train_loss += loss.data.item() * local_batch.size(0)
        num_train_correct  += (torch.max(output, 1)[1] == local_labels).sum().item()
        num_train_examples += local_batch.shape[0]

        train_acc = num_train_correct / num_train_examples
        train_loss  = train_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss       = 0.0
    num_val_correct  = 0
    num_val_examples = 0
    with torch.set_grad_enabled(False):
        for local_batch, local_labels in val_loader:
            # Transfer to GPU
            if use_cuda:
                local_batch, local_labels = local_batch.to(device), local_labels.to(device)
            output = model(local_batch)
            loss = loss_fn(output, local_labels)

            val_loss += loss.data.item() * local_batch.size(0)
            num_val_correct  += (torch.max(output, 1)[1] == local_labels).sum().item()
            num_val_examples += local_batch.shape[0]

        val_acc  = num_val_correct / num_val_examples
        val_loss = val_loss / len(val_loader.dataset)

        if val_loss < min_val_loss:
            epochs_no_improve = 0
            min_val_loss = val_loss
        else:
            epochs_no_improve += 1
  # Check early stopping condition
        if epochs_no_improve == n_epochs_stop:
            early_stop = print('Early stopping!')
            break

        
    print(f'Epoch {epoch+1}/{max_epochs}, train loss: {train_loss:5.4f}, train acc: {train_acc:5.3f}, val loss: {val_loss:5.3f}, val acc: {val_acc:5.3f}')
    
    
    history['loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['acc'].append(train_acc)
    history['val_acc'].append(val_acc)
    
    if early_stop:
        print("Stopped")
        break

RuntimeError: size mismatch, m1: [80 x 23232], m2: [1200 x 128] at /Users/distiller/project/conda/conda-bld/pytorch_1579022061893/work/aten/src/TH/generic/THTensorMath.cpp:136

In [ ]:
local_batch.device

In [ ]:
torch.save(model.state_dict(), "../trained_models/lr1e4_f5_sgd_yeoorder_190comps")

In [ ]:
model.eval()
num_correct  = 0
num_examples = 0
with torch.set_grad_enabled(False):
    for local_batch, local_labels in test_loader:
        # Transfer to GPU
        if use_cuda:
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        output = model(local_batch)
        num_correct  += (torch.max(output, 1)[1] == local_labels).sum().item()
        num_examples += local_batch.shape[0]
            
    test_acc  = num_correct / num_examples

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history['acc'])
plt.plot(history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
test_acc